In [3]:
import pandas as pd
df = pd.read_csv('../Dataset/countries.csv')
df_new = df.drop(['Country Code', '2015 Rank', '2014 Rank', '2013 Rank'], axis = 1)
df_new

,Country Name,2015 Score,2014 Score,2013 Score
0,Taiwan,78,67.0,42.0
1,United Kingdom,76,97.0,94.0
2,Denmark,70,83.0,87.0
3,Colombia,68,66.0,0.0
4,Finland,67,73.0,72.0
...,...,...,...,...
144,Armenia,0,0.0,0.0
145,Belarus,0,0.0,0.0
146,Uzbekistan,0,0.0,0.0
147,Uganda,0,0.0,0.0


# Stacking column

In [4]:
df_final = df_new.rename(columns={"2015 Score": "2015", "2014 Score": "2014", "2013 Score": "2013"})
df_score = df_final.set_index(['Country Name'])

In [5]:
df_score.columns.name = 'Year'
dfs = df_score.stack(dropna = False)
dfs.name = 'Scores'
dfs_new = dfs.reset_index()

Repeat stacking for RANK

In [6]:
rf_new = df.drop(['Country Code', '2015 Score', '2014 Score', '2013 Score'], axis = 1)
rf_final = rf_new.rename(columns={"2015 Rank": "2015", "2014 Rank": "2014", "2013 Rank": "2013"})
rf_rank = rf_final.set_index(['Country Name'])

In [7]:
rf_rank.columns.name = 'Year'
dfr = rf_rank.stack(dropna = False)
dfr.name = 'Rank'
dfr_new = dfr.reset_index()

Merging two dataframe

In [8]:
new_df = pd.merge(dfs_new,dfr_new, how = 'left', left_on=['Country Name','Year'], right_on = ['Country Name','Year'] )
new_df

,Country Name,Year,Scores,Rank
0,Taiwan,2015,78.0,1.0
1,Taiwan,2014,67.0,11.0
2,Taiwan,2013,42.0,36.0
3,United Kingdom,2015,76.0,2.0
4,United Kingdom,2014,97.0,1.0
...,...,...,...,...
442,Uganda,2014,0.0,98.0
443,Uganda,2013,0.0,61.0
444,"Venezuela, Bolivarian Republic of",2015,0.0,123.0
445,"Venezuela, Bolivarian Republic of",2014,0.0,98.0


In [9]:
from pathlib import Path  
filepath = Path('../Dataset/df_opendata.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
new_df.to_csv(filepath) 

# Country unmatch entries

In [10]:
# loading corruption index dataset and list countries
corr = pd.read_csv('../Dataset/merged_cpi_data.csv')
country_cpi = corr['Country'].unique()

# loading open data index dataset
opdata = pd.read_csv('../Dataset/df_opendata.csv')
country_odi = opdata['Country Name'].unique()

a = set(str(x) for x in country_cpi)
b = set(str(x) for x in country_odi) 

In [12]:
joint_items = a.union(b)
unmatched_cpi = a - b
unmatched_odi = b - a
unmatched = unmatched_cpi.union(unmatched_odi)
unmatched

{'Afghanistan',
 'Angola',
 'Antigua and Barbuda',
 'Aruba',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bolivia, Plurinational State of',
 'Brunei',
 'Burundi',
 'Cabo Verde',
 'Cape Verde',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Comoros',
 'Congo',
 'Cuba',
 'Democratic Republic of the Congo',
 'Djibouti',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'FYR Macedonia',
 'Gabon',
 'Gambia',
 'Gibraltar',
 'Guernsey',
 'Guinea-Bissau',
 'Honduras',
 'Iran',
 'Iran, Islamic Republic of',
 'Isle of Man',
 'Jersey',
 'Kiribati',
 'Korea, Republic of',
 'Laos',
 'Liberia',
 'Macau',
 'Macedonia',
 'Macedonia, the Former Yugoslav Republic of',
 'Madagascar',
 'Malawi',
 'Maldives',
 'Mauritania',
 'Mauritius',
 'Moldova',
 'Moldova, Republic of',
 'Mongolia',
 'Montenegro',
 'Montserrat',
 'Mozambique',
 'Namibia',
 'Nicaragua',
 'North Korea',
 'North Macedonia',
 'Papua New Guinea',
 'Republic of the Congo',
 'Russia',
 'Russian Federation',
 'Saint Kitts and Nev

Here we remarked United states and United States of America written differently yet refering to the same countries

In [15]:
#Replacing value in open data index dataset
opdata_new = opdata.replace(['Bolivia, Plurinational State of','Macedonia, the Former Yugoslav Republic of','Iran, Islamic Republic of','Korea, Republic of','Moldova, Republic of', 'Russian Federation','St. Vincent & the Grenadines','Tanzania, United Republic of', 'United States','Venezuela, Bolivarian Republic of' ],['Bolivia','Macedonia','Iran','South Korea','Moldova','Russia','Saint Vincent and the Grenadines','Tanzania','United States of America','Venezuela'])

In [16]:
from pathlib import Path  
filepath = Path('../Dataset/df_opendata_new.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
opdata_new.to_csv(filepath) 

# Combining 2 dataset CPI and ODI 

In [17]:
corr = pd.read_csv('../Dataset/merged_cpi_data.csv')
corr_1 = corr[['Year','Country','CPI Score','Rank']]
df_cpi = corr_1.rename(columns={"Rank": "CPI Rank"})

In [18]:
opendata = pd.read_csv('../Dataset/df_opendata_new.csv')
opdata = opendata[['Country Name','Year','Scores','Rank']]
df_odi = opdata.rename(columns={"Country Name": "Country", "Scores": "ODI Scores",  "Rank": "ODI Rank" })

In [19]:
combined_df = pd.merge(df_cpi,df_odi, how = 'outer')
combined_df

,Year,Country,CPI Score,CPI Rank,ODI Scores,ODI Rank
0,2010,Denmark,93.0,1.0,NaN,NaN
1,2010,New Zealand,93.0,1.0,NaN,NaN
2,2010,Singapore,93.0,1.0,NaN,NaN
3,2010,Finland,92.0,4.0,NaN,NaN
4,2010,Sweden,92.0,4.0,NaN,NaN
...,...,...,...,...,...,...
1817,2014,Gibraltar,NaN,NaN,NaN,NaN
1818,2013,Gibraltar,NaN,NaN,NaN,NaN
1819,2015,"Virgin Islands, British",NaN,NaN,0.0,123.0
1820,2014,"Virgin Islands, British",NaN,NaN,NaN,NaN


In [20]:
from pathlib import Path  
filepath = Path('../Dataset/df_combined.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
combined_df.to_csv(filepath) 

# Grouping by region

predownload csv list countries and region from : 

In [21]:
region = pd.read_csv('../Dataset/group_region.csv')

In [22]:
df_combined = pd.read_csv('../Dataset/df_combined.csv')

Filtering rows that contains only both CPI and ODI scores and discard the rest

In [23]:
df_combined_new = df_combined[(df_combined['CPI Score'].notnull()) & (df_combined['ODI Scores'].notnull())]
df_combined_new.drop(columns=['Unnamed: 0']).reset_index(drop=True)

,Year,Country,CPI Score,CPI Rank,ODI Scores,ODI Rank
0,2013,Denmark,91.0,1.0,87.0,2.0
1,2013,New Zealand,91.0,1.0,66.0,9.0
2,2013,Finland,89.0,3.0,72.0,7.0
3,2013,Sweden,89.0,3.0,67.0,8.0
4,2013,Norway,86.0,5.0,76.0,4.0
...,...,...,...,...,...,...
371,2015,Haiti,17.0,158.0,0.0,123.0
372,2015,Venezuela,17.0,158.0,0.0,123.0
373,2015,Iraq,16.0,161.0,11.0,109.0
374,2015,Libya,16.0,161.0,6.0,120.0


Join the two dataset

In [24]:
df_group = df_combined_new.set_index('Country').join(region.set_index('Country'))
df_group.drop(columns=['Unnamed: 0'])

,Year,CPI Score,CPI Rank,ODI Scores,ODI Rank,Code,Region 1,Region 2,Continent
Country,,,,,,,,,
Albania,2013,31.0,116.0,0.0,61.0,ALB,Southern Europe,NaN,Europe
Albania,2014,33.0,110.0,0.0,98.0,ALB,Southern Europe,NaN,Europe
Albania,2015,36.0,88.0,42.0,37.0,ALB,Southern Europe,NaN,Europe
Algeria,2013,36.0,94.0,0.0,61.0,DZA,Northern Africa,NaN,Africa
Algeria,2014,36.0,100.0,0.0,98.0,DZA,Northern Africa,NaN,Africa
...,...,...,...,...,...,...,...,...,...
Zambia,2014,38.0,85.0,27.0,76.0,ZMB,Eastern Africa,Sub-Saharan Africa,Africa
Zambia,2015,38.0,76.0,0.0,123.0,ZMB,Eastern Africa,Sub-Saharan Africa,Africa
Zimbabwe,2013,21.0,157.0,0.0,61.0,ZWE,Eastern Africa,Sub-Saharan Africa,Africa


In [25]:
from pathlib import Path  
filepath = Path('../Dataset/df_final.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df_group.to_csv(filepath) 